## Importing libraries

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Data Preprocessing

### Training Image preprocessing

In [16]:
training_set=tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,  #32 training examples will be fed to neural network at a time
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

Found 70295 files belonging to 38 classes.


### Validation Image Processing

In [17]:
validation_set=tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

Found 17572 files belonging to 38 classes.


In [ ]:
# training_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [ ]:
# for x,y in training_set:
#     print(x,x.shape)
#     print(y,y.shape)
#     break

tf.Tensor(
[[[[168.75 157.75 155.75]
   [207.5  196.5  194.5 ]
   [191.25 180.25 178.25]
   ...
   [170.75 159.75 157.75]
   [165.75 154.75 152.75]
   [167.5  156.5  154.5 ]]

  [[179.   168.   166.  ]
   [180.   169.   167.  ]
   [176.75 165.75 163.75]
   ...
   [176.   165.   163.  ]
   [178.25 167.25 165.25]
   [172.25 161.25 159.25]]

  [[186.25 175.25 173.25]
   [174.   163.   161.  ]
   [192.75 181.75 179.75]
   ...
   [173.25 162.25 160.25]
   [177.25 166.25 164.25]
   [178.75 167.75 165.75]]

  ...

  [[171.5  163.5  161.5 ]
   [176.5  168.5  166.5 ]
   [178.75 169.75 170.75]
   ...
   [ 94.5   90.5  104.5 ]
   [ 98.5   94.5  108.5 ]
   [ 97.5   93.5  107.5 ]]

  [[178.   170.   168.  ]
   [184.75 176.75 174.75]
   [176.5  167.5  168.5 ]
   ...
   [ 86.5   84.5   97.5 ]
   [ 89.75  87.75 100.75]
   [ 90.    88.   101.  ]]

  [[172.75 167.75 164.75]
   [182.5  174.5  172.5 ]
   [181.   172.   173.  ]
   ...
   [ 78.    76.    90.  ]
   [ 80.5   78.5   92.5 ]
   [ 81.5   79.5   9